In [186]:
import pandas as pd
import altair as alt
import persist_ext as PR
import numpy as np

In [3]:
df_raw = pd.read_csv("avalanches_raw.csv")
print(df_raw.shape)
df_raw.head()

(7563, 25)


,Date,Region,Place,Trigger,Trigger: additional info,Weak Layer,Depth,Width,Vertical,Aspect,...,Buried - Fully,Injured,Killed,Accident and Rescue Summary,Terrain Summary,Weather Conditions and History,Comments 1,Comments 2,Comments 3,Comments 4
0,11/9/2012,Salt Lake,Sunset Peak,Snowboarder,Unintentionally Triggered,New Snow/Old Snow Interface,"14""",80',30',North,...,NaN,NaN,NaN,NaN,NaN,NaN,While it was a small avalanche that was I caug...,NaN,NaN,NaN
1,11/10/2012,Salt Lake,Pioneer Ridge,Skier,Remotely Triggered,Facets,2',20',125',North,...,NaN,NaN,NaN,NaN,NaN,NaN,Directly above cat track leading to lift. Cat...,NaN,NaN,NaN
2,11/10/2012,Salt Lake,Pioneer Bowl,Natural,NaN,Facets,NaN,NaN,NaN,North,...,NaN,NaN,NaN,NaN,NaN,NaN,"Looked and felt like winter today, nice to be ...",NaN,NaN,NaN
3,11/11/2012,Salt Lake,Patsy Marly,Skier,Unintentionally Triggered,New Snow/Old Snow Interface,2.5',100',100',North,...,NaN,NaN,NaN,NaN,NaN,NaN,A North facing aspect with an exposed ridge in...,NaN,NaN,NaN
4,11/11/2012,Salt Lake,Two Dogs,Skier,Remotely Triggered,Facets,3',70',450',North,...,NaN,NaN,NaN,NaN,NaN,NaN,Remotely triggered all the new storm snow (abo...,NaN,NaN,Additional comments by another backcountry obs...


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7563 entries, 0 to 7562
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Date                            7533 non-null   object 
 1   Region                          7520 non-null   object 
 2   Place                           7522 non-null   object 
 3   Trigger                         7304 non-null   object 
 4   Trigger: additional info        2873 non-null   object 
 5   Weak Layer                      4546 non-null   object 
 6   Depth                           5370 non-null   object 
 7   Width                           5696 non-null   object 
 8   Vertical                        3862 non-null   object 
 9   Aspect                          7207 non-null   object 
 10  Elevation                       6836 non-null   object 
 11  Coordinates                     5954 non-null   object 
 12  Caught                          62

In [31]:
raw_col_count = df_raw.count().sort_values(ascending=False)
raw_col_count

Date                              7533
Place                             7522
Region                            7520
Trigger                           7304
Aspect                            7207
Comments 1                        6840
Elevation                         6836
Coordinates                       5954
Width                             5696
Depth                             5370
Weak Layer                        4546
Vertical                          3862
Trigger: additional info          2873
Comments 4                        1053
Carried                            695
Caught                             622
Comments 2                         451
Accident and Rescue Summary        247
Comments 3                         243
Buried - Partly                    206
Buried - Fully                     129
Killed                             114
Injured                             94
Terrain Summary                     56
Weather Conditions and History      50
dtype: int64

In [32]:
threshold = 3000

In [33]:
col_count_5k = raw_col_count[raw_col_count > threshold]
col_count_5k

Date           7533
Place          7522
Region         7520
Trigger        7304
Aspect         7207
Comments 1     6840
Elevation      6836
Coordinates    5954
Width          5696
Depth          5370
Weak Layer     4546
Vertical       3862
dtype: int64

In [52]:
df_5k = df_raw[col_count_5k.index]
df_5k.loc[:, "Weak Layer"] = df_5k["Weak Layer"].fillna("Unknown")
df_5k.loc[:, "Trigger"] = df_5k["Trigger"].fillna("Unknown")
df_5k

,Date,Place,Region,Trigger,Aspect,Comments 1,Elevation,Coordinates,Width,Depth,Weak Layer,Vertical
0,11/9/2012,Sunset Peak,Salt Lake,Snowboarder,North,While it was a small avalanche that was I caug...,"10,400'","40.577977000000, -111.595817000000",80',"14""",New Snow/Old Snow Interface,30'
1,11/10/2012,Pioneer Ridge,Salt Lake,Skier,North,Directly above cat track leading to lift. Cat...,"9,900'",NaN,20',2',Facets,125'
2,11/10/2012,Pioneer Bowl,Salt Lake,Natural,North,"Looked and felt like winter today, nice to be ...","9,700'",NaN,NaN,NaN,Facets,NaN
3,11/11/2012,Patsy Marly,Salt Lake,Skier,North,A North facing aspect with an exposed ridge in...,"9,700'","40.592619000000, -111.616099000000",100',2.5',New Snow/Old Snow Interface,100'
4,11/11/2012,Two Dogs,Salt Lake,Skier,North,Remotely triggered all the new storm snow (abo...,"10,200'","40.599291000000, -111.642315000000",70',3',Facets,450'
...,...,...,...,...,...,...,...,...,...,...,...,...
7558,5/9/2023,Stairs Gulch,Salt Lake,Natural,Northwest,\r\n\r\nI will start off by saying this has ta...,"9,600'","40.607179217054, -111.729240404660",NaN,10',Unknown,NaN
7559,5/21/2023,Red Pine Gulch,Salt Lake,Skier,Northeast,Wet loose slide triggered by skier. Carried sk...,"10,700'","40.540143672049, -111.688926411910",20',"4""",Unknown,NaN
7560,5/20/2023,Salt Lake,Salt Lake,Natural,Southeast,Transitioning from my skins to start boot pack...,"10,700'","40.591020502471, -111.708812700810",25',"2""",Unknown,NaN
7561,6/6/2023,Provo,Provo,Skier,Northwest,I was skiing down the Couloir and decided to m...,"10,900'","40.419867397058, -111.672924744200",15',NaN,Unknown,NaN


In [146]:
df_non_null = df_5k.dropna()
df_non_null

,Date,Place,Region,Trigger,Aspect,Comments 1,Elevation,Coordinates,Width,Depth,Weak Layer,Vertical
0,11/9/2012,Sunset Peak,Salt Lake,Snowboarder,North,While it was a small avalanche that was I caug...,"10,400'","40.577977000000, -111.595817000000",80',"14""",New Snow/Old Snow Interface,30'
3,11/11/2012,Patsy Marly,Salt Lake,Skier,North,A North facing aspect with an exposed ridge in...,"9,700'","40.592619000000, -111.616099000000",100',2.5',New Snow/Old Snow Interface,100'
4,11/11/2012,Two Dogs,Salt Lake,Skier,North,Remotely triggered all the new storm snow (abo...,"10,200'","40.599291000000, -111.642315000000",70',3',Facets,450'
5,11/11/2012,Emma Ridges,Salt Lake,Skier,Southeast,Impressive fast powder cloud ran in front of t...,"10,200'","40.598313000000, -111.628304000000",50',"18""",New Snow,500'
6,11/11/2012,Sunset Peak,Salt Lake,Skier,North,Three of us toured from Brighton to low saddle...,"10,400'","40.578590000000, -111.595087000000","1,500'",3.5',Facets,800'
...,...,...,...,...,...,...,...,...,...,...,...,...
7519,4/22/2023,Cardiff Bowl,Salt Lake,Unknown,East,We spent the day skiing the southerly-facing a...,"9,800'","40.592721660567, -111.649613218710",60',"8""",New Snow/Old Snow Interface,150'
7527,4/22/2023,"Miller Bowl, East",Logan,Snowmobiler,North,"Not sure about the story here, but we observed...","8,700'","41.886233332343, -111.645074831510",45',"18""",New Snow/Old Snow Interface,400'
7530,4/22/2023,Millville Peak,Logan,Snowboarder,North,Details are a bit limited and we're not sure w...,"8,900'","41.677564539953, -111.718065248970",300',"12""",New Snow/Old Snow Interface,600'
7550,5/7/2023,Red Top Mountain,Salt Lake,Natural,West,Saw this avalanche around 9.30 AM from the top...,"10,800'","40.546874131921, -111.663880335390",250',6',Unknown,"1,000'"


In [175]:
def convert_inches_feet_to_inches(x):
    if "'" in x:
        return float(x[:-1].replace(",", "")) * 12
    elif "\"" in x:
        return float(x[:-1].replace(",", ""))
    else:
        print(x)
    return x

df_converted = df_non_null.copy(deep=True)
df_converted.loc[:, "Vertical"] = df_non_null["Vertical"].apply(convert_inches_feet_to_inches)
df_converted.loc[:, "Elevation"] = df_non_null["Elevation"].apply(convert_inches_feet_to_inches) / 12
df_converted.loc[:, "Width"] = df_non_null["Width"].apply(convert_inches_feet_to_inches)
df_converted.loc[:, "Depth"] = df_non_null["Depth"].apply(convert_inches_feet_to_inches)
df_converted = df_converted[['Date', 'Region', 'Place', 'Trigger', 'Weak Layer', 'Vertical', 'Elevation', 'Aspect', 'Width', 'Depth']]
df_converted = df_converted.reset_index(drop=True)

In [176]:
PR.PersistTable(df_converted)

PersistWidget(data_values=[{'__id_column': '1', 'Date': 1352419200000, 'Region': 'Salt Lake', 'Place': 'Sunset…

In [180]:
df_cols_fixed.head()

,Date,Region,Weak Layer,Vertical_inches,Elevation_feet,Aspect,Trigger,Place,Width_inches,Depth_inches,__annotations
0,2012-11-09,Salt Lake,New Snow/Old Snow Interface,360,10400,North,Snowboarder,Sunset Peak,960,14,No Annotation
1,2012-11-11,Salt Lake,New Snow/Old Snow Interface,1200,9700,North,Skier,Patsy Marly,1200,30,No Annotation
2,2012-11-11,Salt Lake,Facets,5400,10200,North,Skier,Two Dogs,840,36,No Annotation
3,2012-11-11,Salt Lake,New Snow,6000,10200,Southeast,Skier,Emma Ridges,600,18,No Annotation
4,2012-11-11,Salt Lake,Facets,9600,10400,North,Skier,Sunset Peak,18000,42,No Annotation


In [182]:
df_cols_fixedd = df_cols_fixed.drop(columns=["__annotations"])
df_cols_fixedd.head()

,Date,Region,Weak Layer,Vertical_inches,Elevation_feet,Aspect,Trigger,Place,Width_inches,Depth_inches
0,2012-11-09,Salt Lake,New Snow/Old Snow Interface,360,10400,North,Snowboarder,Sunset Peak,960,14
1,2012-11-11,Salt Lake,New Snow/Old Snow Interface,1200,9700,North,Skier,Patsy Marly,1200,30
2,2012-11-11,Salt Lake,Facets,5400,10200,North,Skier,Two Dogs,840,36
3,2012-11-11,Salt Lake,New Snow,6000,10200,Southeast,Skier,Emma Ridges,600,18
4,2012-11-11,Salt Lake,Facets,9600,10400,North,Skier,Sunset Peak,18000,42


In [263]:
def make_outliers(df, cols, limits=None, n=3, _k=5, direction='both'):
    random_indices = np.random.choice(df.index, size=n, replace=False)
    
    for id, col in enumerate(cols):
        if col in df.columns:
            mean = df[col].mean()
            std = df[col].std()
            o_type = str(df[col].dtype)

            
            for idx in random_indices:
                k = np.random.randint(_k, _k * 2)
                if direction == 'higher':
                    outlier_value = mean + k * std
                elif direction == 'lower':
                    outlier_value = mean - k * std
                elif direction == 'both':
                    # Randomly choose to make the outlier higher or lower
                    direction_choice = np.random.choice(['higher', 'lower'])
                    multiplier = k if direction_choice == 'higher' else -k
                    outlier_value = mean + multiplier * std
                else:
                    raise ValueError("Direction must be 'higher', 'lower', or 'both'.")

                print(limits, id)
                if limits is not None and id < len(limits):
                    limit = limits[id]
                    if limit is not None:
                        outlier_value = min(outlier_value, limit)
                
                if "Int" in o_type:
                    df.at[idx, col] = int(outlier_value)
                else:
                    df.at[idx, col] = outlier_value
                df.loc[idx, "Moved"] = True

    return df
    return df
ddf = df_cols_fixedd.copy(True)
ddf["Moved"] = False

ddf = make_outliers(ddf, ["Vertical_inches", "Elevation_feet"], limits=[100000], direction="higher", n=5, _k=8)

ddf = pd.read_csv("avalanches_data_processed_old_orig.csv")
alt.Chart(ddf).mark_point().encode(x="Elevation_feet", y="Vertical_inches", color="Moved:N", opacity=alt.condition("datum.Moved", alt.value(0.8), alt.value(0.4)))

[100000] 0
[100000] 0
[100000] 0
[100000] 0
[100000] 0
[100000] 1
[100000] 1
[100000] 1
[100000] 1
[100000] 1


alt.Chart(...)